# Nomoto model first order PIT

In [ ]:
# %load imports.py
"""
These is the standard setup for the notebooks.
"""

%matplotlib inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
#plt.style.use('paper')

import copy

import numpy as np
import os

from src.data import database
from mdldb import mdl_to_evaluation
from mdldb.tables import Run
import src.data
import os.path

from sklearn.pipeline import Pipeline

import sympy as sp
from sklearn.metrics import r2_score
import src.reporting.paper_writing as paper_writing

from src.equations import equations
from src.equations import symbols
from rolldecayestimators.substitute_dynamic_symbols import lambdify

In [ ]:
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from pandas_profiling import ProfileReport
import evaluation.evaluation_helpers as evaluation_helpers
from scipy.optimize import least_squares

## Nomotos equation:

In [ ]:
Math(vlatex(equations.nomoto_first_order))

In [ ]:
db = database.get_db()

In [ ]:
sql="""
SELECT * from run
INNER JOIN projects
ON run.project_number==projects.project_number
    INNER JOIN loading_conditions
    ON (run.loading_condition_id == loading_conditions.id)
        INNER JOIN models
        ON run.model_number == models.model_number
            INNER JOIN ships
            ON models.ship_name == ships.name

WHERE run.test_type=="spiral"     
"""
data = pd.read_sql_query(sql=sql, con=db.engine)
data = data.loc[:,~data.columns.duplicated()]

In [ ]:
data.describe()

In [ ]:
#profile = ProfileReport(statistics, title='Pandas Profiling Report')
#profile.to_widgets()

In [ ]:
loading_conditions = data.groupby(by=['loading_condition_id','ship_speed'])

In [ ]:
loading_conditions.describe()

In [ ]:
loading_condition = loading_conditions.get_group(name=(3,19))
#loading_condition = loading_conditions.get_group(name=(144,16))

In [ ]:
loading_condition.describe()

### Load all data for one loading condition

In [ ]:
df_all = pd.DataFrame()
interesting_columns = ['delta','x0','y0','z0','phi','theta','psi']
for index, run in loading_condition.iterrows():
    db_run = db.session.query(Run).get(int(run.id))
    df = database.load_run(db_run=db_run)
    df['t'] = df.index 
    
    df_=evaluation_helpers.coord(df=df)  # add psi and position etc.
    
    df = pd.concat((df,df_), axis=1)

    df['run_id'] = run.id
    
    df_all = df_all.append(df[['t','run_id'] + interesting_columns], ignore_index=True)

In [ ]:
df_all.describe()

In [ ]:
fig,ax=plt.subplots()
runs = df_all.groupby(by='run_id')
for run_id, df in runs:
    df['x0']-=df.iloc[0]['x0']
    df['y0']-=df.iloc[0]['y0']
    
    df.plot(x='y0',y='x0', ax=ax)
    
ax.get_legend().remove()
ax.set_aspect('equal', 'box')

In [ ]:
def derivate(group):
    df = group.set_index('t')
    
    ddf = np.gradient(df, df.index, axis=0).mean(axis=0)
    s = pd.Series(ddf, index=df.columns)
    
    return s

In [ ]:
df = runs.mean()

ddf = runs.apply(func= derivate)

df['u']=ddf['x0']
df['v']=ddf['y0']
df['w']=ddf['z0']
df['p']=ddf['phi']
df['q']=ddf['theta']
df['r']=ddf['psi']
df.sort_values(by='r', inplace=True)

In [ ]:
fig,ax=plt.subplots()
df.plot(x='delta', y='r', ax=ax, style='o')
ax.grid(True)
ax.set_title('Reverse spiral plot')

In [ ]:
print(loading_condition.iloc[0]['project_path'])

In [ ]:
spiral_eq = sp.simplify(equations.nomoto_first_order.subs(symbols.r_1d,0))
Math(vlatex(spiral_eq))

In [ ]:
r_lambda=lambdify(sp.solve(spiral_eq,symbols.r)[0])

In [ ]:
def residual(parameters, X, ys):
    r = r_lambda(*parameters,delta=X['delta'])
    
    error = r - ys
    return error

initial_guess = [-1,]

kwargs={
    'X':df,
    'ys':df['r'],
}

result = least_squares(fun=residual, x0=initial_guess, kwargs=kwargs, method='lm')
parameters={
    'K':result.x,
}

In [ ]:
r_predict = r_lambda(**parameters,delta=df['delta'])

fig,ax=plt.subplots()
df.plot(x='delta', y='r', ax=ax, style='o')
ax.plot(df['delta'],r_predict, 'b-')
ax.grid(True)
ax.set_title('Reverse spiral plot');

In [ ]:
K_3 = sp.symbols('K_3')
delta_0 = sp.symbols('delta_0')


spiral_eq_3 = sp.Eq(symbols.delta,
      sp.solve(spiral_eq,symbols.delta)[0] + symbols.r**5/K_3 + delta_0) 

Math(vlatex(spiral_eq_3))

In [ ]:
A, A_3 = sp.symbols('A A_3')
spiral_eq_3_A = spiral_eq_3.subs([(symbols.K,1/A),
                  (K_3,1/A_3),
                 ])

In [ ]:
delta_lambda_3=lambdify(sp.solve(spiral_eq_3_A,symbols.delta)[0])

In [ ]:
delta_lambda_3

In [ ]:
np.random.seed()

def residual_3(parameters, X, ys):
    delta = delta_lambda_3(*parameters,r=X['r'])
    error = (delta - ys)**2
    return error

initial_guess = [-0.1,-1000,0]

kwargs={
    'X':df,
    'ys':df['delta'],
}

bounds = ([-np.inf,-np.inf],
          [0,0,np.inf])   

result = least_squares(fun=residual_3, x0=initial_guess, kwargs=kwargs, max_nfev=1000, 
                       loss='linear', f_scale=0.1, method='lm')
parameters_3={
    'A':result.x[0],
    'A_3':result.x[1],
    'delta_0':result.x[2],
}

In [ ]:
result

In [ ]:
parameters_3

In [ ]:
N=100
r=np.linspace(df['r'].min(),df['r'].max(),N)
delta_predict = delta_lambda_3(**parameters_3,r=r)

fig,ax=plt.subplots()
df.plot(x='delta', y='r', ax=ax, style='o')
ax.plot(delta_predict,r, 'b-')
ax.grid(True)
ax.set_title('Reverse spiral plot');

In [ ]:
from scipy import polyval, polyfit
df['r**5'] = df['r']**5
X = df[['r','r**5']].copy()
X['1']=1.0
x, residuals, rank, s = np.linalg.lstsq(X, df['delta'], rcond=None)
parameters_4 = {
    'A':x[0],
    'A_3':x[1],
    'delta_0':x[2],
}

delta_predict = delta_lambda_3(**parameters_4,r=r)

fig,ax=plt.subplots()
df.plot(x='delta', y='r', ax=ax, style='o')
ax.plot(delta_predict,r, 'b-')
ax.grid(True)
ax.set_title('Reverse spiral plot');